In [1]:
from bs4 import BeautifulSoup
import urllib2
import re
import dateutil.parser
import numpy as np
import pickle
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
os.environ
import time
import pandas as pd
import csv
import sklearn
import matplotlib.pyplot as plt


#Supervised Learning Challenge


#Challenge 1

In [2]:

voting_data_set = urllib2.urlopen('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data').read()
with open('voting_data_set.csv','w') as f:
    pickle.dump(voting_data_set, f)
        

In [3]:
with open('voting_data_set.pkl','rb') as input_file:
    voting_file = pickle.load(input_file)

In [4]:
voting_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', header = -1)

In [5]:
voting_df = voting_df.replace ('y', 1).replace('n', 0)
voting_df  = voting_df.replace ('?',np.nan)

In [6]:
list_mean = list(voting_df.mean(axis=0, skipna=True))

In [7]:
for i in range(1,17):
    
    voting_df[i] = voting_df[i].replace(np.nan,list_mean[i-1])

In [8]:
type(voting_df)

pandas.core.frame.DataFrame

#Challenge 2

In [9]:
from sklearn.cross_validation import train_test_split


In [10]:
training_set, test_set = train_test_split(voting_df)

In [11]:
#training_set

#Challenge 3

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [13]:
x_train = training_set.iloc[:, 1:]
y_train = training_set.iloc[:, 0]
x_test = test_set.iloc[:, 1:]
y_test = test_set.iloc[:,0]

In [14]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=3, p=2, weights='uniform')

In [15]:
y_pred = neigh.predict(x_test)
accuracy_score(y_test,y_pred)

0.96330275229357798

In [16]:
def find_best_number_of_neighbor(train_data, test_data, N):
    x_train = train_data.iloc[:, 1:]
    y_train = train_data.iloc[:, 0]
    x_test =  test_data.iloc[:, 1:]
    y_test = test_data.iloc[:, 0]
    best_k = 1
    best_score = 0
    for i in range(1,N+1):
        neigh = KNeighborsClassifier(n_neighbors=i)
        neigh.fit(x_train,y_train)
        new_score = accuracy_score(y_test,neigh.predict(x_test))
        if new_score > best_score:
            best_score = new_score
            best_k = i
    
    return best_k, best_score
        
        

In [17]:
def neighbor_test_score(train_data, test_data, N):
    x_train = train_data.iloc[:, 1:]
    y_train = train_data.iloc[:, 0]
    x_test =  test_data.iloc[:, 1:]
    y_test = test_data.iloc[:, 0]
    neigh = KNeighborsClassifier(n_neighbors=N)
    neigh.fit(x_train,y_train)
    return accuracy_score(y_test,neigh.predict(x_test))
    

In [18]:
find_best_number_of_neighbor(training_set,test_set, 20)

(3, 0.96330275229357798)

#Challenge 4

In [19]:
from sklearn.linear_model import LogisticRegression
from statsmodels.formula.api import Logit

In [20]:
logreg = LogisticRegression()

In [21]:
def log_reg_test(train_data, test_data):
    x_train = train_data.iloc[:, 1:]
    y_train = train_data.iloc[:, 0]
    x_test =  test_data.iloc[:, 1:]
    y_test = test_data.iloc[:, 0]
    logreg = LogisticRegression()
    logreg.fit(x_train,y_train)
    return accuracy_score(y_test, logreg.predict(x_test))
    

In [22]:
logreg.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [23]:
accuracy_score(y_test, logreg.predict(x_test))

0.98165137614678899

#Challenge 5

In [24]:
a = dict(voting_df[0].value_counts())
a

{'democrat': 267, 'republican': 168}

In [25]:
import pylab as pl
X = np.arange(len(a))
pl.bar(X, a.values(), align='center', width=0.5)
pl.xticks(X, a.keys())
ymax = max(a.values()) + 1
pl.ylim(0, ymax)
pl.show()

In [26]:
def fake_democrat_pred(array_or_matrix):
    array_length = len(array_or_matrix)
    return ['democrat']*array_length

In [27]:
#fake_democrat_pred(x_test)

In [28]:
accuracy_score(y_test,fake_democrat_pred(x_test))

0.65137614678899081

In [29]:
def fake_republican_pred(array_or_matrix):
    array_length = len(array_or_matrix)
    return ['republican']*array_length

In [30]:
accuracy_score(y_test,fake_republican_pred(x_test))

0.34862385321100919

#Challenge 6

In [31]:
def accuracies_plot(train_data,test_data, test_name, k):
    x_train = train_data.iloc[:, 1:]
    y_train = train_data.iloc[:, 0]
    x_test =  test_data.iloc[:, 1:]
    y_test = test_data.iloc[:, 0]
    accuracies_list = []
    for i in range(1,k+1):
        if test_name == 'fake_democrat_pred':
            score = accuracy_score(y_test,
                                   fake_democrat_pred(x_test))
        elif test_name == 'fake_republican_pred':
            score = accuracy_score(y_test,
                                   fake_republican_pred(x_test))   
        elif test_name == 'log_reg_test':
            score = log_reg_test(train_data,test_data)
        elif test_name == 'neighbor_test_score':
            score = neighbor_test_score(train_data,test_data, i)
            
        accuracies_list.append(score)
    
    plt.plot(range(1,k+1),accuracies_list)
    #return accuracies_list

            
            
                 
          


In [32]:
plt.figure()
accuracies_plot(training_set,test_set, 'fake_democrat_pred' ,20)
accuracies_plot(training_set,test_set, 'fake_republican_pred' ,20)
accuracies_plot(training_set,test_set, 'log_reg_test' ,20)
accuracies_plot(training_set,test_set, 'neighbor_test_score' ,20)
plt.show()

#Challenge 7

In [33]:
from sklearn.learning_curve import learning_curve

In [34]:
X = voting_df.iloc[:, 1:]
y = voting_df.iloc[:, 0]



In [35]:
m, train_err, test_err = learning_curve(logreg,X,y)
train_cv_err = np.mean(train_err,axis=1)
test_cv_err = np.mean(test_err,axis=1)
plt.plot(m, train_cv_err)
plt.plot(m, test_cv_err)
plt.show()

In [36]:
neigh = KNeighborsClassifier(n_neighbors=4)
m, train_err, test_err = learning_curve(neigh,X,y)
train_cv_err = np.mean(train_err,axis=1)
test_cv_err = np.mean(test_err,axis=1)
plt.plot(m, train_cv_err)
plt.plot(m, test_cv_err)
plt.show()

#Challenge 8

In [37]:
from sklearn.naive_bayes import GaussianNB

In [38]:
model = GaussianNB()
model.fit(x_train,y_train)
accuracy_score(y_test, model.predict(x_test))

0.95412844036697253

In [39]:
from sklearn.svm import SVC
svc_model = SVC()
svc_model.fit(x_train,y_train)
accuracy_score(y_test,svc_model.predict(x_test))

0.96330275229357798

In [40]:
from sklearn.tree import DecisionTreeClassifier
tree_model = DecisionTreeClassifier()
tree_model.fit(x_train,y_train)
accuracy_score(y_test,tree_model.predict(x_test))

0.93577981651376152

In [41]:
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier()
forest_model.fit(x_train,y_train)
accuracy_score(y_test,forest_model.predict(x_test))

0.98165137614678899

# Challenge 9

In [42]:
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(model, X,y)

In [43]:
Gauss_score = np.mean(scores)
Gauss_score

0.93103448275862066

In [44]:
svc_score = np.mean(cross_val_score(svc_model,X,y))
svc_score

0.95402298850574718

In [45]:
tree_score = np.mean(cross_val_score(tree_model,X,y))
tree_score

0.9517241379310345

In [46]:
forest_score = np.mean(cross_val_score(forest_model,X,y))
forest_score

0.96551724137931039

#Challenge 10

In [47]:
vote_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', header = -1)

In [48]:
vote_df = vote_df.replace('y',1).replace('n',0).replace('?',np.nan)

In [49]:
list_vote_mean = list(vote_df.mean())

In [50]:
for i in range(1,17):
    if list_vote_mean[i-1] >= 0.5:
        vote_df[i] = vote_df[i].replace(np.nan,1)
    else:
        vote_df[i] = vote_df[i].replace(np.nan,0)
     

In [51]:
y_value = vote_df.iloc[:,15]
x_value = pd.concat([vote_df.iloc[:,1:15],vote_df.iloc[:,16]],axis=1)

In [52]:
cross_val_score(logreg,x_value,y_value)

array([ 0.80689655,  0.77241379,  0.72413793])

In [53]:
np.mean(cross_val_score(logreg,x_value,y_value))

0.76781609195402289

#Challenge 11

In [54]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/haberman/haberman.data'
breast_cancer_data = pd.read_csv(url,header = -1)

In [55]:
breast_cancer_data.rename(columns={0:'Ages',1:'Year_of_operation',
                                   2:'Nums_positive_nodes',3:'Survival_status'}, 
                          inplace=True) 
breast_cancer_data

,Ages,Year_of_operation,Nums_positive_nodes,Survival_status
0,30,64,1,1
1,30,62,3,1
2,30,65,0,1
3,31,59,2,1
4,31,65,4,1
5,33,58,10,1
6,33,60,0,1
7,34,59,0,2
8,34,66,9,2
9,34,58,30,1


In [56]:
breast_cancer_data.Ages.std()

10.803452349303276

In [57]:
survival_patient = breast_cancer_data.loc[breast_cancer_data['Survival_status']==1]

In [58]:
print survival_patient.Ages.mean()
print survival_patient.Ages.std()

52.0177777778
11.0121541799


In [59]:
dead_patient = breast_cancer_data.loc[breast_cancer_data['Survival_status']==2]

In [60]:
print dead_patient.Ages.mean()
print dead_patient.Ages.std()

53.6790123457
10.1671372083


In [61]:
min(breast_cancer_data['Year_of_operation'])

58

In [62]:
max(breast_cancer_data['Year_of_operation'])

69

In [65]:
cancer_train_set, cancer_test_set= train_test_split(breast_cancer_data)


In [97]:
x_cancer_data = breast_cancer_data.iloc[:,:3]
y_cancer_data = breast_cancer_data.iloc[:,3]

In [80]:
x_cancer = cancer_train_set.iloc[:,:3]
y_cancer = cancer_train_set.iloc[:,3]

In [86]:
x_test_cancer = cancer_test_set.iloc[:,:3]
y_test_cancer = cancer_test_set.iloc[:,3]

In [85]:
logreg.fit(x_cancer,y_cancer)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [87]:
accuracy_score(y_test_cancer,logreg.predict(x_test_cancer))

0.77922077922077926

In [88]:
#Logistic Regression does not do so well in predicting survival

In [104]:
print logreg.coef_
print logreg.intercept_

[[ 0.01988529 -0.03534786  0.08967194]]
[-0.20594719]


In [100]:
m_cancer, cancer_train_err, cancer_test_err = learning_curve(logreg,x_cancer_data,y_cancer_data) 

In [101]:
cancer_train_err = np.mean(cancer_train_err,axis=1)
cancer_test_err = np.mean(cancer_test_err,axis=1)

In [102]:
plt.plot(m,cancer_train_err)
plt.plot(m,cancer_test_err)
plt.show()

In [106]:
x_cancer_data.head()

,Ages,Year_of_operation,Nums_positive_nodes
0,30,64,1
1,30,62,3
2,30,65,0
3,31,59,2
4,31,65,4


In [107]:
# So as the logistics regression function look like this:
# F = 1/(1+exp(-(-0.205 + 0.019885*ages -0.0354786* year_of_operation +0.08967194*nums_positive_nodes)))
# or log(F(x)/(1-F(x))) = -0.205 + 0.019885*ages -0.0354786* year_of_operation +0.08967194*nums_positive_nodes
# so increasing in ages and number of positive nodes will increase your odd in having cancer.